In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score 
n_estimators = 20
max_depth = 3


In [2]:
def train_model():
    data = load_iris()
    X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42 )
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42 )
    model.fit(X_train, y_train)


    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')

    return model, accuracy, precision, recall, f1

In [3]:

import mlflow
import mlflow.sklearn



def log_model_to_mlflow(model, accuracy,precision, recall, f1,  n_estimators=10, max_depth=3):
    mlflow.set_tracking_uri(uri='http://127.0.0.1:5000/')
    mlflow.set_experiment('iris classifier / v2')

    with mlflow.start_run():
        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('max_depth', max_depth)

        mlflow.log_metric('accuracy', accuracy)
        mlflow.log_metric('precision', precision)
        mlflow.log_metric('f1', f1)
        mlflow.log_metric('recall', recall)

        mlflow.sklearn.log_model(model, 'random_forest_model')

        model_uri = f"runs:/{mlflow.active_run().info.run_id}/model"
        print('model_uri : ', model_uri)

        model_name = 'random_forest_iris'
        mlflow.register_model(model_uri=model_uri, name=model_name)

    from mlflow.tracking import MlflowClient

    client = MlflowClient()


    client.transition_model_version_stage(
        name = model_name, 
        version = 1, 
        stage = "archived"

    )


In [4]:
model, accuracy, precision, recall, f1 = train_model()


In [5]:
log_model_to_mlflow(model, accuracy, precision, recall, f1)

2024/11/14 19:38:53 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2024/11/14 19:38:54 WARNING mlflow.models.model: Model logged without a signature and input example. Ple

model_uri :  runs:/dda158514a1a494d9b660a12175c71f8/model


/tmp/ipykernel_18376/2943856903.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
